In [1]:
from neo4j import GraphDatabase
uri = "bolt://localhost:7687"
pwd = "password"

driver = GraphDatabase.driver(uri, auth=("neo4j", pwd))

query = "MATCH (n) RETURN COUNT(n)"

with driver.session() as session:
    result = session.run(query)
    print(result.single()["COUNT(n)"])

0


In [2]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///manufacturers.csv' AS line
CREATE (:Manufacturer {name: line.name, mfgID: line.mfgID})
"""

with driver.session() as session:
    result = session.run(query)

In [3]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///motors.csv' AS line
CREATE (:Motor {
  commonName: line.commonName,
  delays: line.delays,
  diameter: toFloat(line.diameter),
  infoUrl: line.infoUrl,
  totImpulseNs: toFloat(line.totImpulseNs),
  manufacturer: line.manufacturer,
  burnTimeS: toFloat(line.burnTimeS),
  propInfo: line.propInfo,
  length: toFloat(line.length),
  avgThrustN: toFloat(line.avgThrustN),
  dataFiles: line.dataFiles,
  impulseClass: line.impulseClass,
  sparky: line.sparky,
  caseInfo: line.caseInfo,
  propWeightG: toFloat(line.propWeightG),
  certOrg: line.certOrg,
  motorId: line.motorId,
  availability: line.availability,
  maxThrustN: toFloat(line.maxThrustN),
  totalWeightG: toFloat(line.totalWeightG),
  designation: line.designation,
  updatedOn: line.updatedOn,
  type: line.type,
  mfgID: line.mfgID
})
"""

with driver.session() as session:
    result = session.run(query)

In [4]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///kits.csv' AS row
MERGE (kit:Kit {uniqueID: row.UniqueID})
ON CREATE SET
  kit.name = row.Name,
  kit.url = row.url,
  kit.imageSrc = row['image_src'],
  kit.recommendedEngines = row['Recommended Engines'],
  kit.projectedMaxAltitude = row['Projected Max Altitude'],
  kit.recoverySystem = row['Recovery System'],
  kit.length = toFloat(row.Length),
  kit.diameter = toFloat(row.Diameter),
  kit.estimatedWeight = toFloat(row['Estimated Weight']),
  kit.estimatedAssemblyTime = row['Estimated Assembly Time'],
  kit.finMaterials = row['Fin Materials'],
  kit.decalType = row['Decal Type'],
  kit.launchSystem = row['Launch System'],
  kit.launchRodSize = row['Launch Rod Size'],
  kit.instructions = row.instructions,
  kit.ageRecommendation = row['Age Recommendation'],
  kit.mfgID = row.mfgID,
  kit.height = toFloat(row.Height),
  kit.weight = toFloat(row.Weight),
  kit.motorMount = row['Motor Mount'],
  kit.parachuteSize = row['Parachute Size'],
  kit.shockCordType = row['Shock Cord Type'],
  kit.shockCordMount = row['Shock Cord Mount'],
  kit.finThickness = toFloat(row['Fin Thickness']),
  kit.ringThickness = toFloat(row['Ring Thickness']),
  kit.instructionsUrl = row.Instructions,
  kit.price = toFloat(row.Price),
  kit.currency = row.Currency,
  kit.sku = row.SKU,
  kit.stockStatus = row['Stock Status'],
  kit.description = row.Description,
  kit.links = row.Links,
  kit.parachute = row.Parachute,
  kit.finArray = row['Fin Array'],
  // Using CASE to conditionally set the complexity property
  kit.complexity = CASE WHEN row.Complexity IS NOT NULL THEN row.Complexity ELSE NULL END;
"""

with driver.session() as session:
    result = session.run(query)

In [5]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///rockets.csv' AS line
CREATE (:Rocket {id: line.id, name: line.name, mfgID: line.mfgID, photo: line.photo})
"""

with driver.session() as session:
    result = session.run(query)

In [6]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///models.csv' AS line
CREATE (:Model:Rocket {name: line.name, id: line.id, photo: line.photo})
"""

with driver.session() as session:
    result = session.run(query)

In [7]:
query = """
CREATE (:Person {
email: 'ronaldhatcher@mac.com',
family_name: 'Hatcher',
given_name: 'Ronald',
id: 'kp_878e41a7d93846cca23114ad549e8126',
picture: 'https://lh3.googleusercontent.com/a/ACg8ocKicc19o8E_h-mOGCLngSlTsFxJLcIGONz_ntc4RW5_3Q=s96-c'})
"""

with driver.session() as session:
    result = session.run(query)

In [8]:
query = """
LOAD CSV WITH HEADERS FROM 'file:///designs.csv' AS line
CREATE (:Design {id: line.id, rocket: line.rocket, name: line.name, filename: line.filename, fileURL: line.fileURL})
"""

with driver.session() as session:
    result = session.run(query)

Add our relationships


In [9]:
query = """
MATCH (manufacturer:Manufacturer), 
      (motor:Motor) 
WHERE motor.mfgID = manufacturer.mfgID 
MERGE (manufacturer)-[:MAKES]->(motor);
"""
with driver.session() as session:
    result = session.run(query)

In [10]:
query = """
MATCH (manufacturer:Manufacturer), 
(kit:Kit)
WHERE kit.mfgID = manufacturer.mfgID 
MERGE (manufacturer)-[:MAKES]->(kit);
"""
with driver.session() as session:
    result = session.run(query)

In [11]:
query = """
MATCH (manufacturer:Manufacturer), 
(rocket:Rocket)
WHERE rocket.mfgID = manufacturer.mfgID 
MERGE (manufacturer)-[:MAKES]->(rocket);
"""
with driver.session() as session:
    result = session.run(query)

In [12]:
query = """
match (r:Rocket {mfgID: 'vorb', id: 'launcherone'}),
      (m:Model {id: 'lone'})
      MERGE (m)-[:BASED_ON]->(r)
"""
with driver.session() as session:
    result = session.run(query)

In [13]:
query = """
match (r:Rocket {mfgID: 'hughes', id: 'aim54a'}),
      (m:Model {id: 'phx'})
      MERGE (m)-[:BASED_ON]->(r)
"""
with driver.session() as session:
    result = session.run(query)

In [14]:
query = """
MATCH (k:Kit {uniqueID: '9e295916-5309-4471-847d-4d42c814a806'}),
      (m:Model {id: 'phx'})
      MERGE (m)-[:BASED_ON]->(k)
"""
with driver.session() as session:
    result = session.run(query)

In [15]:
query = """
MATCH (p:Person {id: "kp_878e41a7d93846cca23114ad549e8126"}), (m:Model)
WHERE m.id IN ['lone', 'phx', 'bfhr']
MERGE (p)-[:OWNS]->(m)
"""
with driver.session() as session:
    result = session.run(query)

In [16]:
query = """
MATCH (r:Rocket), (d:Design)
WHERE r.id = d.rocket
MERGE (r)-[:DEFINED_BY]->(d)
"""

with driver.session() as session:
    result = session.run(query)